# Chain of Responsibility
>Avoid couping the sender of a request to its receiver by giving more than one object a chance to handle the request. Chain the receiving objects and pass the request along the chain until an object handles it.

Sender does not know which object should receive its message beforehand.

## Problem
In the code below `Sender` already knows at coding time that it will be calling `ReceiverOne` inside its `do` method. What if the `Sender` has no way of knowing this ahead of time. In fact even at runtime it does not know which `Receiver` is the right receiver to call? Only the receivers know whether they can fulfill the request that `Sender` is sending them.

In [5]:
class ReceiverOne:
    def action_one(self, arg):
        print(f"ReceiverOne::action {arg}")
        

class ReceiverTwo:
    def action_two(self, arg):
        print(f"ReceiverTwo::action {arg}")
        
        
class Sender:
    def __init__(self, recvr: ReceiverOne):
        self.recvr: ReceiverOne = recvr
    
    def do(self, arg):
        print(f"Sending with {arg}")
        self.recvr.action_one(arg)

In [6]:
recvr1 = ReceiverOne()
sender = Sender(recvr1)
sender.do("hello")

Sending with hello
ReceiverOne::action hello


## Solution
Define a `Handler` interface for handling requests. This interface should also give the next handler in the chain. All receiver classes must now implement this `Handler` interface. The `Sender` is given the first handler in the chain and that is the only one it hands off the request to. It does not know which handler this is.

In [20]:
from abc import ABC, abstractmethod

class Handler(ABC):
    def __init__(self, successor: "Handler"):
        self._successor: Handler = successor
        
    @abstractmethod
    def handle(self, arg):
        pass
    
    @property
    def successor(self):
        return self._successor
    

class ReceiverOne(Handler):
    def __init__(self, successor: Handler):
        super().__init__(successor)
        
    def handle(self, arg):
        if arg == "hello":
            self.action_one(arg)
        elif self.successor:
            self.successor.handle(arg)
        else:
            raise RuntimeError("Could not find the right receiver!")

    def action_one(self, arg):
        print(f"ReceiverOne::action {arg}")
        

class ReceiverTwo(Handler):
    def __init__(self, successor: Handler):
        super().__init__(successor)
        
    def handle(self, arg):
        if arg == "world":
            self.action_two(arg)
        elif self.successor:
            self.successor.handle(arg)
        else:
            raise RuntimeError("Could not find the right receiver!")
            
    def action_two(self, arg):
        print(f"ReceiverTwo::action {arg}")
        
        
class Sender:
    def __init__(self, handler):
        self.handler = handler
    
    def do(self, arg):
        print(f"Sending with {arg}")
        self.handler.handle(arg)

In [22]:
handler = ReceiverOne(ReceiverTwo(None))
sender = Sender(handler)
sender.do("hello")
sender.do("world")

Sending with hello
ReceiverOne::action hello
Sending with world
ReceiverTwo::action world
